In [94]:
import asyncio
import uuid
from google.adk import Agent
from google.adk.events import Event
from google.genai.types import Content, Part
from google.adk.sessions import Session
from typing import Dict, List

print("Build 4 proven patterns that deliver consistent results")

try:
    test_agent = Agent(name="tester", model="llama3.2:3b")
    # Create a single session with required fields
    adk_session = Session(
        id=str(uuid.uuid4()),
        appName="test-app",
        userId="test-user-123"
    )
    # Manually attach the session to itself to satisfy the agent's internal context requirement
    adk_session.session = adk_session
    
    print("Agent and its session object initialized successfully.")
except Exception as e:
    print(f"Error initializing agent: {e}")

Build 4 proven patterns that deliver consistent results
Error initializing agent: "Session" object has no field "session"


## Effective Patterns

In [92]:
class ProvenPromptPatterns:
    @staticmethod
    def chain_of_thought(problem: str, context: str = "") -> str:
        return f"""
PROBLEM: {problem}
CONTEXT: {context}

Think step-by-step:
1. Break down the problem
2. Analyze components
3. Synthesize solution

Provide reasoning for each step.
""".strip()
    
    @staticmethod
    def role_based(task: str, role: str, expertise: str = "") -> str:
        return f"""
You are an expert {role}.
Expertise: {expertise}

TASK: {task}

Provide expert recommendations with:
1. Professional best practices
2. Industry standards
3. Practical solutions
""".strip()
    
    @staticmethod
    def template_based(content: str, format_template: str) -> str:
        return f"""
CONTENT: {content}

OUTPUT FORMAT:
{format_template}

Follow the exact structure above.
""".strip()
    
    @staticmethod
    def step_by_step(goal: str, process: str) -> str:
        return f"""
GOAL: {goal}
PROCESS: {process}

Execute systematically:
- State what you're doing
- Show your work
- Verify completion

Provide detailed execution.
""".strip()

patterns = ProvenPromptPatterns()
print("Proven Patterns loaded!")

Proven Patterns loaded!


## Real Scenarios

In [93]:
async def main(agent, validator, session):
    print("Agent received. Running tasks by passing the full Session object...")

    # Test Chain-of-Thought
    cot_prompt = patterns.chain_of_thought("Our customer churn rate has increased by 15% in the last quarter.", "We need to understand the root causes and devise a retention strategy.")
    agent.generate_content_config = {"temperature": 0.3}
    prompt_content = Content(parts=[Part(text=cot_prompt)])
    context_event = Event(content=prompt_content, author="user")
    session.events.append(context_event)
    response_chunks = []
    async for chunk in agent.run_async(session):
        session.events.append(chunk)
        if hasattr(chunk, 'content') and hasattr(chunk.content, 'text'):
            response_chunks.append(chunk.content.text)
    cot_response = "".join(response_chunks)
    print(f"Chain-of-Thought Response: {len(cot_response)} chars")

    # Test Role-Based
    role_prompt = patterns.role_based("Recommend a scalable cloud architecture for a new social media app.", "Senior Cloud Architect", "AWS, Kubernetes, Microservices")
    agent.generate_content_config = {"temperature": 0.5}
    prompt_content = Content(parts=[Part(text=role_prompt)])
    context_event = Event(content=prompt_content, author="user")
    session.events.append(context_event)
    response_chunks = []
    async for chunk in agent.run_async(session):
        session.events.append(chunk)
        if hasattr(chunk, 'content') and hasattr(chunk.content, 'text'):
            response_chunks.append(chunk.content.text)
    role_response = "".join(response_chunks)
    print(f"Role-Based Response: {len(role_response)} chars")

    # Test Template-Based
    template_prompt = patterns.template_based(
        content="User-reported issues: Login failures, slow dashboard loading, data export errors.",
        format_template="""Subject: [Summarize the Main Issue]
Dear [User Name],
We are addressing the issues you reported: [List of Issues].
Our team will take these actions: [List of Actions].
Update ETA: [Provide an ETA]."""
    )
    agent.generate_content_config = {"temperature": 0.1}
    prompt_content = Content(parts=[Part(text=template_prompt)])
    context_event = Event(content=prompt_content, author="user")
    session.events.append(context_event)
    response_chunks = []
    async for chunk in agent.run_async(session):
        session.events.append(chunk)
        if hasattr(chunk, 'content') and hasattr(chunk.content, 'text'):
            response_chunks.append(chunk.content.text)
    template_response = "".join(response_chunks)
    print(f"Template-Based Response: {len(template_response)} chars")

    # Test Step-by-Step
    step_prompt = patterns.step_by_step(
        goal="Contain and resolve a security breach.",
        process="Follow this incident response framework: 1. Containment 2. Threat Assessment 3. Investigation 4. Recovery 5. Post-Incident Communication"
    )
    agent.generate_content_config = {"temperature": 0.2}
    prompt_content = Content(parts=[Part(text=step_prompt)])
    context_event = Event(content=prompt_content, author="user")
    session.events.append(context_event)
    response_chunks = []
    async for chunk in agent.run_async(session):
        session.events.append(chunk)
        if hasattr(chunk, 'content') and hasattr(chunk.content, 'text'):
            response_chunks.append(chunk.content.text)
    step_response = "".join(response_chunks)
    print(f"Step-by-Step Response: {len(step_response)} chars")
    
    print("\n--- Running Validation ---")
    
    test_cases = [
        ("Chain-of-Thought", cot_response, ["step", "analysis", "churn"]),
        ("Role-Based", role_response, ["architecture", "scalable", "recommend"]),
        ("Template-Based", template_response, ["subject", "issue", "actions"]),
        ("Step-by-Step", step_response, ["contain", "assess", "recover"])
    ]

    scores = []
    for name, response, expected in test_cases:
        metrics = validator.validate_response(response, expected)
        score = validator.calculate_score(metrics)
        scores.append(score)
        print(f"{name}: {score:.2f} score")

    avg_score = sum(scores) / len(scores)
    print(f"\nAverage pattern score: {avg_score:.2f} ({avg_score*100:.1f}%)")


# Pass the agent, validator, and session into the main function
await main(test_agent, validator, adk_session)

Agent received. Running tasks by passing the full Session object...


AttributeError: 'Session' object has no attribute 'session'

## Automated Validation System

In [ ]:
class PromptValidator:
    def validate_response(self, response: str, expected: List[str] = None) -> Dict[str, float]:
        metrics = {}
        
        # Completeness
        metrics["completeness"] = min(1.0, len(response) / 500) if len(response) > 200 else 0.3
        
        # Structure
        structure_count = sum(1 for ind in [":", "-", "1.", "\n"] if ind in response)
        metrics["structure"] = min(1.0, structure_count / 4)
        
        # Relevance
        if expected:
            found = sum(1 for elem in expected if elem.lower() in response.lower())
            metrics["relevance"] = found / len(expected)
        else:
            metrics["relevance"] = 0.8
        
        # Actionability
        action_words = ["recommend", "suggest", "should", "action", "step"]
        action_count = sum(1 for word in action_words if word in response.lower())
        metrics["actionability"] = min(1.0, action_count / 3)
        
        return metrics
    
    def calculate_score(self, metrics: Dict[str, float]) -> float:
        weights = {"completeness": 0.2, "relevance": 0.4, "structure": 0.2, "actionability": 0.2}
        return sum(metrics[key] * weights[key] for key in weights)

validator = PromptValidator()

# Validate all responses
test_cases = [
    ("Chain-of-Thought", cot_response, ["step", "analysis", "churn"]),
    ("Role-Based", role_response, ["architecture", "scalable", "recommend"]),
    ("Template-Based", template_response, ["subject", "issue", "actions"]),
    ("Step-by-Step", step_response, ["contain", "assess", "recover"])
]

scores = []
for name, response, expected in test_cases:
    metrics = validator.validate_response(response, expected)
    score = validator.calculate_score(metrics)
    scores.append(score)
    print(f"{name}: {score:.2f} score")

avg_score = sum(scores) / len(scores)
print(f"\nAverage pattern score: {avg_score:.2f} ({avg_score*100:.1f}%)")

C:\Python313\Lib\ast.py:54: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


NameError: name 'cot_response' is not defined

## A/B Testing

In [ ]:
class PromptABTester:
    def __init__(self, validator):
        self.validator = validator
        self.results = []
    
    async def run_test(self, agent, prompt_a, prompt_b, test_name, expected=None):
        response_a = await agent.run(prompt_a)
        response_b = await agent.run(prompt_b)
        
        score_a = self.validator.calculate_score(self.validator.validate_response(response_a, expected))
        score_b = self.validator.calculate_score(self.validator.validate_response(response_b, expected))
        
        improvement = ((score_b - score_a) / score_a) * 100 if score_a > 0 else 0
        
        result = {
            "test": test_name,
            "score_a": score_a,
            "score_b": score_b,
            "improvement": improvement
        }
        self.results.append(result)
        
        print(f"\n{test_name}:")
        print(f"A: {score_a:.3f}, B: {score_b:.3f}, Improvement: {improvement:+.1f}%")
        return result

ab_tester = PromptABTester(validator)

# Test 1: Basic vs Chain-of-Thought
basic = "Our app has 2.1 stars. Users complain about crashes. Create action plan."
enhanced = patterns.chain_of_thought("App has 2.1 stars, crashes reported", "Mobile app performance issues")

await ab_tester.run_test(test_agent, basic, enhanced, "Basic vs CoT", ["action", "plan", "crashes"])

# Test 2: Generic vs Role-Based  
generic = "Choose between React Native, Flutter, or native. Recommend best option."
role_enhanced = patterns.role_based("Choose mobile framework", "Mobile Architect", "Cross-platform expertise")

await ab_tester.run_test(test_agent, generic, role_enhanced, "Generic vs Role", ["react", "flutter", "recommend"])

# Calculate summary
improvements = [r["improvement"] for r in ab_tester.results]
avg_improvement = sum(improvements) / len(improvements)
print(f"\nAverage improvement: {avg_improvement:+.1f}%")

if avg_improvement >= 20:
    print("Excellent: 20%+ consistent gains!")
else:
    print("Good: Meaningful improvements shown")

## Production Library

In [ ]:
class ProductionLibrary:
    def __init__(self):
        self.patterns = ProvenPromptPatterns()
        self.usage = {"cot": 0, "role": 0, "template": 0, "step": 0}
    
    def get_best_pattern(self, task_type):
        mapping = {
            "analysis": "cot",
            "expert": "role", 
            "communication": "template",
            "process": "step"
        }
        return mapping.get(task_type, "cot")
    
    def create_prompt(self, task_type, **kwargs):
        pattern = self.get_best_pattern(task_type)
        self.usage[pattern] += 1
        
        if pattern == "cot":
            return self.patterns.chain_of_thought(kwargs.get("problem", ""), kwargs.get("context", ""))
        elif pattern == "role":
            return self.patterns.role_based(kwargs.get("task", ""), kwargs.get("role", ""), kwargs.get("expertise", ""))
        elif pattern == "template":
            return self.patterns.template_based(kwargs.get("content", ""), kwargs.get("format", ""))
        elif pattern == "step":
            return self.patterns.step_by_step(kwargs.get("goal", ""), kwargs.get("process", ""))

library = ProductionLibrary()

# Test automatic pattern selection
scenarios = [
    ("analysis", "Market analysis task"),
    ("expert", "Technical architecture"), 
    ("communication", "Customer email"),
    ("process", "Security response")
]

print("INTELLIGENT PATTERN SELECTION:")
for task_type, description in scenarios:
    recommended = library.get_best_pattern(task_type)
    print(f"{description}: {recommended}")

# Generate some example prompts
market_prompt = library.create_prompt(
    "analysis", 
    problem="Competitor launched at 50% lower price",
    context="Enterprise B2B market"
)

tech_prompt = library.create_prompt(
    "expert",
    task="Design rate limiting for 1M requests/hour", 
    role="Backend Engineer",
    expertise="Microservices, Redis"
)

print(f"\nGenerated market prompt: {len(market_prompt)} chars")
print(f"Generated tech prompt: {len(tech_prompt)} chars")
print(f"\nUsage stats: {library.usage}")
print("Production library working perfectly!")